# Loading Radar Data

In [ ]:
# Import Necessary Libraries
import os
import numpy as np
from datetime import datetime
import pysteps
# Import utility functions from the repository (custom functions for data loading and other functionalities
import utility   
import matplotlib.pyplot as plt
from pprint import pprint
from pysteps import io, nowcasts, rcparams
from pysteps.motion.lucaskanade import dense_lucaskanade
from pysteps.postprocessing.ensemblestats import excprob
from pysteps.utils import conversion, transformation
from pysteps.visualization import plot_precip_field


root_path = os.getcwd()
# Define the data directory for the repository data
data_path = os.path.join(root_path, "Data")
# Define metadata files
metadata_X = utility.get_matadata(os.path.join(data_path, "radarmappatipo.tif"), type="X")
folders = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder))]
events = [event for event in os.listdir(os.path.join(data_path,
                                                     'UNICA_SG')) if os.path.isdir(os.path.join(data_path,
                                                                                                'UNICA_SG', event))]
# Define parameters for X-band radar data
data_source_X = 'UNICA_SG'
event_subdir = events[3]
f_ext_X = "png"
num_prev_files = 3
num_next_files = 6
timestep = 5
# Choose the date for particular subset
eventdates = [eventdate for eventdate in os.listdir(os.path.join(data_path,
                                                                 'UNICA_SG',
                                                                 event_subdir)) if os.path.isfile(os.path.join(data_path,
                                                                                                               'UNICA_SG', 
                                                                                                               event_subdir,
                                                                                                               eventdate))]
# extract the date in datetime format from the filename
date = datetime.strptime('20191222_0620.png', "%Y%m%d_%H%M.png")
# Load X-band radar data
R_dn, metadata_dn = utility.import_files_by_date(date, 
                                           data_path, 
                                           data_source_X,
                                           event_subdir, 
                                           f_ext_X, 
                                           metadata_X,
                                           num_prev_files,
                                           0,
                                           timestep)


# Noise removal using Watershed technique
R_dn_clean = np.empty_like(R_dn)
for t in range(R_dn.shape[0]):
    R_dn_clean[t, :, :] = utility.noise_remove(R_dn[t, :, :], type="Watershed")

# Convert Digital Number to Reflectivity (dBZ)
R_dbz,metadata_dbz = utility.dn_to_dbz(R_dn_clean,metadata_dn)
# Convert to rain rate
R_R, metadata_R = conversion.to_rainrate(R_dbz, metadata_dbz)
# Log-transform the data to unit of dBR, set the threshold to 0.1 mm/h,
# set the fill value to -15 dBR
R_dbr, metadata_dbr = transformation.dB_transform(R_R, metadata_R, threshold=0.1, zerovalue=-15.0)

# Set missing values with the fill value
R_dbr[~np.isfinite(R_dbr)] = -15.0

# Nicely print the metadata
pprint(metadata_dbr)

# Load observed data to compare with nowcast results

In [ ]:
R_O, metadata_O = utility.import_files_by_date(date, 
                                               data_path, 
                                               data_source_X,
                                               event_subdir, 
                                               f_ext_X, 
                                               metadata_X,
                                               0,
                                               num_next_files,
                                               timestep)
# Noise removal using Watershed technique
R_O_clean = np.empty_like(R_O)
for t in range(R_dn.shape[0]):
    R_O_clean[t, :, :] = utility.noise_remove(R_O[t, :, :], type="Watershed")

# Convert Digital Number to Reflectivity (dBZ)
R_O_dbz,metadata_O_dbz = utility.dn_to_dbz(R_O_clean,metadata_O)
# Convert to rain rate
R_O_R, metadata_O_R = conversion.to_rainrate(R_O_dbz, metadata_O_dbz)

# Estimate the motion field 

In [ ]:
# motion field using lucas kanade optical flow (dense method) with shitomasi tracking methods
from pysteps.visualization import quiver
V1 = dense_lucaskanade(R_dbr,fd_method="shitomasi",verbose=True)
fig, ax = plt.subplots(figsize=(20, 10))

plot_precip_field(
            R_R[-1, :, :],
            ptype="intensity",
            geodata=metadata_R,
            units="mm/h",
            ax=axes[0],
            colorscale="pysteps")
axes[0].set_title(f"Dense LK motion (shitomasi) {metadata_R['timestamps'][-1].strftime('%d/%m/%Y %H:%M')}")
quiver(V1, geodata=metadata_R, step=75,ax=ax)     
utility.plot_modification(ax,metadata_R)

# Deterministic nowcast with SPROG

In [ ]:
# Set nowcast parameters
n_ens_members = 20
n_leadtimes = 6
seed = 24
# The S-PROG nowcast
sprog = nowcasts.get_method("sprog")
R_f = sprog(
    R_dbr[-3:, :, :],
    V1,
    n_leadtimes,
    n_cascade_levels=6,
    R_thr=-10.0
)
# Back-transform to rain rate
R_f = transformation.dB_transform(R_f, threshold=-10.0, inverse=True)[0]

# Deterministic nowcast with semilagrangian extrapolation

In [ ]:
# Extrapolate the last radar observation
extrapolate = nowcasts.get_method("extrapolation")
R_f_sl = extrapolate(R_dbr[-1, :, :], 
                     V1, 
                     n_leadtimes,extrap_method="semilagrangian")

# Back-transform to rain rate
R_f_sl = transformation.dB_transform(R_f_sl, threshold=-10.0, inverse=True)[0]

# Deterministic nowcast with ANVIL

In [ ]:
anvil = nowcasts.get_method("anvil")
R_f_A =  anvil(R_R, 
               V1, 
               n_leadtimes, 
               ar_window_radius=25, 
               ar_order=2)

# Deterministic nowcast with Eulerian persistence

In [ ]:
# Extrapolate the last radar observation
extrapolate = nowcasts.get_method("extrapolation")
R_f_el = extrapolate(R_dbr[-1, :, :], 
                     V1, 
                     n_leadtimes,
                     extrap_method="eulerian")

# Back-transform to rain rate
R_f_el = transformation.dB_transform(R_f_el, threshold=-10.0, inverse=True)[0]

# Function to plot nowcasting frames

In [ ]:
# Time indices corresponding to t-15 min, t-10 min, t-5 min, t min
input_indices = [0,1,2,3]
# Time indices corresponding to t+5 min, t+10 min, t+15 min and t+30 min
output_indices = [0,1,2,5]
# subplot for single dataset
def plot_radar_data(data, metadata, time_indices, text):
    fig, axes = plt.subplots(1, len(time_indices), figsize=(30, 10))
    for i, idx in enumerate(time_indices):
        ax = axes[i]
        plot_precip_field(
            data[idx, :, :],
            ptype="intensity",
            geodata=metadata,
            units="mm/h",
            ax=ax,
            colorscale="pysteps"
        )
        ax.set_title(f"{text} {metadata['timestamps'][idx].strftime('%d/%m/%Y %H:%M')}")
        utility.plot_modification(ax,metadata)
    plt.tight_layout()
    plt.show()

# Plot each nowcasts

In [ ]:
plot_radar_data(R_R, metadata_R, input_indices, "Input ")

In [ ]:
plot_radar_data(R_O_R, metadata_O_R, output_indices, "Observed ")

In [ ]:
plot_radar_data(R_f_el, metadata_O_R, output_indices, "Persistence ")

In [ ]:
plot_radar_data(R_f_sl, metadata_O_R, output_indices, "Extrapolation ")

In [ ]:
plot_radar_data(R_f, metadata_O_R, output_indices, "SPROG  ")

In [ ]:
plot_radar_data(R_f_A, metadata_O_R, output_indices, "ANVIL  ")

# Evaluation matrices among different nowcasting methods